In [12]:
### This script looks at all files
### 1) Checks deleted files for md5 related fields and clears them
###                         for qc metric and changes status of qc metric object to deleted
###                         clears the qc metric field
###                         for workflows and deletes all of them
### 2) Checks other files for workflows and deleted old workflows
###                                     and deleted problematic ones (status or rev)
### takes around 20 min

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def get_wfr_report(wfrs):
    wfr_report = []
    for wfr_data in wfrs:
        wfr_rep = {}
        """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
        wfr_uuid = wfr_data['uuid']
        wfr_data = ff_utils.get_metadata(wfr_uuid, connection = ff)
        wfr_status = wfr_data['run_status']
        try:
            wfr_name = wfr_data['title'].split(' run ')[0]
        except:
            print('ProblematicCase')
            print(wfr_data['uuid'], wfr_data.get('display_title', 'no title'))
            continue
        wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
        run_hours = (datetime.utcnow()-wfr_time).total_seconds()/3600
        wfr_name_list = wfr_data['title'].split(' run ')[0].split('/')
        wfr_name = wfr_name_list[0]
        try:
            wfr_rev = wfr_name_list[1] 
        except:
            wfr_rev = "0"

        output_files = wfr_data.get('output_files',None)
        output_uuids = []
        if output_files:
            for i in output_files:
                if i.get('value', None):
                    output_uuids.append(i['value'])

        wfr_rep = {'wfr_uuid': wfr_data['uuid'],
                   'wfr_status': wfr_data['run_status'],
                   'wfr_name': wfr_name,
                   'wfr_rev': wfr_rev,
                   'wfr_date': wfr_time,
                   'run_time': run_hours,
                   'status': wfr_data['status'],
                   'outputs': output_uuids}
        wfr_report.append(wfr_rep)
    wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
    return wfr_report

    
def printTable(myDict, colList=None):
    """ Pretty print a list of dictionaries Author: Thierry Husson"""
    if not colList: colList = list(myDict[0].keys() if myDict else [])
    myList = [colList] # 1st row = header
    for item in myDict: myList.append([str(item[col] or '') for col in colList])
    colSize = [max(map(len,col)) for col in zip(*myList)]
    formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
    myList.insert(1, ['-' * i for i in colSize]) # Seperating line
    for item in myList: print(formatStr.format(*item))
        


In [13]:
print 'started at', datetime.utcnow()
delete_workflows = raw_input("Do you want to delete old workflowruns (if not, only report will be displayed (y/n))")

# what kind of files should be searched for worflow run inputs, use url compatible naming

# accepted workflows
# workflow name, accepted revision numbers (0 if none), accetable run time (hours)
workflow_details = [
                    ['md5', ['0'], 24],
                    ['fastqc-0-11-4-1', ['1'], 72],
                    ['hi-c-processing-parta-juicer', ['25','26'], 150],
                    ['hi-c-processing-partb', ['31', '34', '38'],150],
                    ['hi-c-processing-partc', ['3','8'], 150],
                    ['add-hic-normvector-to-mcool', ['3'], 150],
                    ['extract-mcool-normvector-for-juicebox', ['0'], 150]
                   ]
deleted_wfr_no = 0
files_with_deleted_wfr = 0
files = [i['uuid'] for i in ff_utils.get_metadata('files' , connection=ff)['@graph']]
print len(files), 'files in the system'
deleted_wfrs = []
counter = 0
del_md5 = 0
del_qc = 0
deleted_output = 0
for a_file in files:
    counter += 1
    if counter % 100 == 0:
        print counter, files_with_deleted_wfr
    raw_file = ff_utils.get_metadata(a_file, connection = ff, frame='embedded')
    deleted_wf = False
    wfr_report = []
    wfrs = raw_file.get('workflow_run_inputs')

    if raw_file['status'] == 'deleted':
        if delete_workflows.lower() in ['y', 'yes']:
            # clean deleted files of md5 and qc metrics
            for a_field in ['content_md5sum', 'md5sum']:  
                if raw_file.get(a_field):
                    ff_utils.delete_field(raw_file, a_field, connection=ff)
                    del_md5 += 1
            if raw_file.get('quality_metric'):
                qc_uuid = raw_file['quality_metric']['uuid']
                ff_utils.delete_field(raw_file, 'quality_metric', connection=ff)
                # delete quality metrics object
                patch_data = {'status': "deleted"}
                ff_utils.patch_metadata(patch_data, obj_id=qc_uuid ,connection=ff)
                del_qc += 1
        # delete all workflows for deleted files
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            for wfr_to_del in wfr_report:
                if wfr_to_del['status'] != 'deleted':
                    deleted_wf = True
                    deleted_wfr_no += 1
                    
                    
                    ## TEMPORARY PIECE
                    if wfr_to_del['status'] == 'released to project':
                        print('saved from deletion',wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue
                    if wfr_to_del['status'] == 'released':
                        print('delete released????',wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue
                        
                        
                        
                        
                        
                    print(wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    if delete_workflows.lower() in ['y', 'yes']:
                        patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                        deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                        # delete output files of the deleted workflow run
                        if wfr_to_del['outputs']:
                            for out_file in wfr_to_del['outputs']:
                                deleted_output += 1
                                ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
       
                
    else:
        # get a report on all workflow_runs
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            # printTable(wfr_report, ['wfr_name', 'run_time', 'wfr_rev', 'run_time', 'wfr_status'])
            for wf_name,accepted_rev,accepted_run_time in workflow_details:
                #for each type of worklow make a list of old ones, and patch status and description
                sub_wfrs = [i for i in wfr_report if i['wfr_name'] == wf_name]
                if sub_wfrs:
                    active_wfr = sub_wfrs[-1]
                    old_wfrs = sub_wfrs [:-1]
                    # check the status of the most recent workflow
                    if active_wfr['wfr_status'] != 'complete':
                        if active_wfr['wfr_status'] == 'running' and active_wfr['run_time'] < accepted_run_time:
                            print wf_name,'still running for', a_file
                        else:
                            old_wfrs.append(active_wfr)
                    elif active_wfr['wfr_rev'] not in accepted_rev:
                        old_wfrs.append(active_wfr)
                    if old_wfrs:
                        for wfr_to_del in old_wfrs:
                            if wfr_to_del['status'] != 'deleted':
                                deleted_wf = True
                                deleted_wfr_no += 1
                                
                                
                                
                                
                                
                                ## TEMPORARY PIECE
                                if wfr_to_del['status'] == 'released to project':
                                    print('saved from deletion',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                if wfr_to_del['status'] == 'released':
                                    print('delete released????',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                
                                
                                
                                
                                
                                print(wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                
                                if delete_workflows.lower() in ['y', 'yes']:
                                    patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                                    deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                                    
                                    ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                                    # delete output files of the deleted workflow run
                                    if wfr_to_del['outputs']:
                                        for out_file in wfr_to_del['outputs']:
                                            deleted_output += 1
                                            ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
    if deleted_wf:
        files_with_deleted_wfr += 1

if delete_workflows.lower() in ['y', 'yes']:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files deleted"
else:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files need to be deleted"

print len(deleted_wfrs)
print del_md5, 'md5 fields deleted'
print del_qc, 'qc metrics deleted'
print deleted_output, 'deleted output files'
print 'finished at', datetime.utcnow()

started at 2017-09-12 19:59:38.538340
Do you want to delete old workflowruns (if not, only report will be displayed (y/n))y
4326 files in the system
100 0
ProblematicCase
(u'd8a60edf-49e5-4c33-afab-9ec90d656777', u'WorkflowRun from 2017-08-10')
ProblematicCase
(u'd8a60edf-49e5-4c33-afab-9ec90d656666', u'WorkflowRunSbg from 2017-08-10')
ProblematicCase
(u'72588c5e-4c52-4e51-bebe-8981b5855b6f', u'WorkflowRunAwsem from 2017-09-07')
(u'md5', 'deleted file workflow', u'd03e7b76-a379-4060-b0a3-65c4e6e8fff9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'423bd803-e9bf-40ac-b42b-97ccc85711c0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b2b7e578-2788-4193-b128-c9ce9778003d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'04de3377-3ccc-4fc0-8daf-71634acbf78c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e66c3019-aa80-4607-90b8-0251d1e9794c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2a5bdde4-9501-4d96-9dda-6bc280319842', u'4DNFIO67AFHV')
(u'md5', 'delet

(u'md5', 'deleted file workflow', u'fac9a9f0-c5b0-4c87-8a9f-e0c74b9e47bf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5f199bb7-8a67-4163-98c8-83bac09ff696', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1b4eb2f7-30a1-48bb-99cd-c06f24ca3e8a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b8b16468-0019-4402-af68-449d638bd00d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f884890e-decb-4456-a711-fae9c81911d7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'99fc12a0-49cd-47aa-ad81-d909c83b3f58', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e83561a3-f735-4a29-a9db-ca07034fec72', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e159086d-e8a6-4371-ad10-08c90345dc48', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8be3065c-5801-4c0a-85d7-16727a8cf47c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a2d1f6be-e81d-4cfd-aa78-49f1796f3b71', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5678f92e-4402-4d38-aed6-9247d50ede91', u'4DN

(u'md5', 'deleted file workflow', u'3d60727f-c3b8-4ee4-baba-147064592cb6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'52e367d6-f2ff-43ff-b89c-95094b257e93', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'97139259-123c-411e-b1a4-7ad746be7116', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fe5e8f86-174f-4aec-b875-237c031eb54f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'34ed0dda-ba21-4368-9db0-c153fef5a789', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'60073bcd-7335-48af-9468-6ac048ff9a3f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'24ca0b60-e9ab-484c-b6d1-7c9b1771226e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'731928b4-e680-4351-9657-b7e5f3329831', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ee696275-b601-4389-9ef4-027e7202ea63', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0b9e1fe5-466f-4bc4-a8a8-e587afeba789', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'7410b5a3-d54c-4c0e-b0ea-c71ddeb91197', u'4DN

(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFIANXLTA3')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIMDUVALM')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIX97V4QW')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIXDYZNGA')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIY4ADY12')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIAI5WWUK')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIRXWQHY8')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIU1MIHUL')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIHCWUJV2')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79f

('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI5H0U2IB')
600 54
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIKKZBHI1')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFILJJ6PMI')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIC6HGO0T')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIASXKH2N')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIVU1YKV3')
700 59
800 59
900 59
1000 59
1100 59
1200 59
1300 59
1400 59
(u'fastqc-0-11-4-1', 'old style or dub', u'd70b4599-658a-4020-9430-1687ac87f973', u'4DNFIK96H3DL')
(u'fastqc-0-11-4-1', 'old style or dub', u'8abcbf20-c2bb-4a8e-b0c

(u'hi-c-processing-partb', 'deleted file workflow', u'e5927906-cf80-46da-8a81-1093926865af', u'4DNFHFYGN09R')
(u'hi-c-processing-partb', 'deleted file workflow', u'5cc47020-2397-46c7-975b-a0c19b2899fd', u'4DNFHFYGN09R')
3000 135
(u'md5', 'old style or dub', u'9de58daf-1c83-4f3d-81e5-e08034a65676', u'4DNFIN1VFFCD')
(u'md5', 'old style or dub', u'fec40be6-f565-405f-9b74-d8754274685e', u'4DNFIQ5NI8TY')
(u'md5', 'old style or dub', u'0961c843-ada5-4abb-8ef1-9f66a286faa7', u'4DNFIQK4MFWL')
(u'md5', 'old style or dub', u'5f0be557-782f-40b7-8448-4e1c3b92c815', u'4DNFI37UIVDH')
(u'md5', 'old style or dub', u'332088d0-7d28-419f-916b-40d1c4a59e5a', u'4DNFIA8HTWVW')
(u'md5', 'old style or dub', u'629bd164-e1d5-4f72-877b-4f7284df3148', u'4DNFIHCMAFAZ')
(u'md5', 'old style or dub', u'521a9000-6782-43db-8b6c-ff770f8c40bf', u'4DNFITVNI13Q')
(u'md5', 'old style or dub', u'be823ac0-d75e-4b09-8ae2-1e58fe71afc9', u'4DNFIZGJG2VU')
(u'md5', 'old style or dub', u'7e98ddcc-8e40-4e5a-8853-edae9f670ce3', u'4DN

In [ ]:
started at 2017-09-12 19:59:38.538340
Do you want to delete old workflowruns (if not, only report will be displayed (y/n))y
4326 files in the system
100 0
ProblematicCase
(u'd8a60edf-49e5-4c33-afab-9ec90d656777', u'WorkflowRun from 2017-08-10')
ProblematicCase
(u'd8a60edf-49e5-4c33-afab-9ec90d656666', u'WorkflowRunSbg from 2017-08-10')
ProblematicCase
(u'72588c5e-4c52-4e51-bebe-8981b5855b6f', u'WorkflowRunAwsem from 2017-09-07')
(u'md5', 'deleted file workflow', u'd03e7b76-a379-4060-b0a3-65c4e6e8fff9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'423bd803-e9bf-40ac-b42b-97ccc85711c0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b2b7e578-2788-4193-b128-c9ce9778003d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'04de3377-3ccc-4fc0-8daf-71634acbf78c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e66c3019-aa80-4607-90b8-0251d1e9794c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2a5bdde4-9501-4d96-9dda-6bc280319842', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'43e4b688-0a60-432a-b401-df08afa4f3ad', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'78495b9c-02ec-4f0d-b0a4-5278755473d0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'db3cdb6a-878b-4b17-95b5-fba82f1f530a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'45dbcd66-ebce-49e7-8483-5932c4d95e5f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd4080458-ff21-44d0-bbf8-e99d0cbfb3ea', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'558ea6a4-f1c8-47a1-ad26-54fa99118f67', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'cf237b0a-2838-48bd-872b-5d137c198d69', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'65f801ca-56d6-4c6d-9300-9a9eb93fb4bf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'362d8bc9-2f7d-4835-96a0-ed2919259201', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'cc1863e9-b056-4091-a270-914310bf8e25', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0d2f5a59-4855-4b65-b26b-f9b97fc88047', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'cfba5697-1537-4615-86e9-7af3e2c4625d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5a7651f3-d5dc-4775-839c-709b1f50bdd9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f0b461ec-9b57-44f5-9406-b3edf5e58238', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'73f27850-6fbd-41e3-8c9f-ee851296fcbf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8a0cac10-02a4-4070-854f-5b203a3fdd89', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1d8e7d2f-4f15-46a1-ae73-d30aad3c0406', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2b79d053-81d0-462a-be28-2d70cddb8bf8', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'576ae863-6b97-4b60-934a-0607fe607816', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2f76c346-84b3-4d79-ba44-eef245363aab', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f3335e00-2657-47b1-9783-61e8ddf61b0d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'41092ef6-c1b9-4525-bff4-6845cf496e51', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'85fbb826-6215-40fa-9f30-976a61dd1305', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'75999607-4627-46dd-954e-a1583bcb40c2', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'be426691-4dca-4ce1-9f05-213376a4f25e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f76f560b-addc-497e-ae62-6dbca197a50a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'edd4ced2-b38b-49ca-a924-b3b3bc045c57', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2430286e-0bbb-4f44-bc72-7373045a7c31', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b03c10e9-134f-4439-be73-6246bb82e540', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'bb46e1cc-479a-4ef5-b0e4-a9a090e3f31e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'dff5d166-dc68-4ef0-9ba2-6dfe0d12e212', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd18190a7-eefe-4996-b7a2-e310feefe4cc', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a7e63c7a-704c-415e-89ff-d704ab486d59', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'4fd67a00-2ec5-45c5-a6fb-3bd96f6b111c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'7dd8562d-ccef-4916-a2b4-96dd63514b4d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ca022af0-ae0f-4465-ba95-01bd923a5423', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd6034b6f-e873-4a7d-a949-182d2228cb29', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'55cde80e-c28f-4f04-b5a0-6fcd8b804017', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8114ed4a-28ec-400e-9a47-dc488dded1ff', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'6c2fd56c-d44f-407a-81cc-7f249b1cb839', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'bf83dfab-5190-4cbd-9160-b4562ce8253c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'014a4e42-1e4c-4f35-9477-2260e3cf7801', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'df480b0e-25f1-49b1-8c02-77cb418cbd7d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2ad59f33-82e4-4c08-bcb6-592d0541463e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2156a3ef-b264-4fb2-a8d6-eed99c85087c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a34dcc95-02bd-471a-86b8-90f8a292e0d7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e3830864-cba0-42cc-8511-671a65d3a226', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2d4baeb8-2a9f-4c4c-9daf-cea1c6a0b2f9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'bdf4935d-4b51-4881-ab41-16399b532650', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fe208b11-c8d8-460b-97b9-afda7d4c1ba6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3d137cf2-984a-4812-8642-3182cb47b9f3', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b0c3d969-bb15-4c9c-bdd1-850a00c1a375', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5d1ab7e4-5ccc-42ec-be0e-e1afbc16723a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'21d3b38e-c790-47e5-b317-36c0a702f952', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'59d9ba33-20d8-4ccd-9c25-409a403cb406', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'005eee83-c214-4adf-963f-e44c978dcdb1', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e07cab47-95d9-469e-b5ab-61f94fc3b79a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2df0a252-3c91-4c5c-a1e3-a6de3b4d55ea', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'9ca5cb0b-158c-4ca6-9168-e481586ac2c1', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1e0fd9b0-3e7a-4aa9-998b-947c6d5ccdf0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'07bd53c9-c657-4c76-bdc9-077e46c85ed8', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'50dea000-7cb7-40d6-9244-1da35f1d4bdf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'9112aad3-cedd-41f6-bfa5-6a0979a065b1', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5ee2f699-5046-42e2-80c0-9a4959f37f83', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd2c5f8f1-e108-4e61-89f5-9fcf352ce44b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'faeaeb10-d6c9-4aec-8016-b9291b2d94ec', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0c397fe4-8f62-4d9c-a1f9-9a893a4c4a25', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0cc3f0b5-de4c-4cab-8ac3-0689bfdcf5b9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'eec982f4-48d9-4dd3-8086-e6fc48e038af', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'44fd2a72-06ae-48fa-8e58-1dd2c512f50d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'7d33935c-ec3b-41ac-b863-703d50f86b51', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'63d59257-e348-4c1c-a551-9bc1cebf5739', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a090872d-bc0e-41fc-beec-304487b18dc7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'69c53487-a9b1-4494-8ade-cc30c609a97c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'02c04743-b6f6-4819-b45b-5e0b41ee752a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e8f2d13e-15d9-405d-bb43-e6d9a1e89c50', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'c3618045-b5e4-4c38-b078-4279023df946', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'47103047-bf32-4fa3-980c-c3610bad9471', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'26d72108-eed2-4886-b247-1d301db8c703', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fac9a9f0-c5b0-4c87-8a9f-e0c74b9e47bf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5f199bb7-8a67-4163-98c8-83bac09ff696', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1b4eb2f7-30a1-48bb-99cd-c06f24ca3e8a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b8b16468-0019-4402-af68-449d638bd00d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f884890e-decb-4456-a711-fae9c81911d7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'99fc12a0-49cd-47aa-ad81-d909c83b3f58', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e83561a3-f735-4a29-a9db-ca07034fec72', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e159086d-e8a6-4371-ad10-08c90345dc48', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8be3065c-5801-4c0a-85d7-16727a8cf47c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a2d1f6be-e81d-4cfd-aa78-49f1796f3b71', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5678f92e-4402-4d38-aed6-9247d50ede91', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e9acc5f9-e9e6-4fad-888e-3799d9682a2f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fe3ccc06-2d61-4f11-9ebf-bfefca29565e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ff104327-4efe-4129-a663-a134a059eeae', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'42c34e1f-bb65-43fe-aae4-11b4f583f39d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'181fe63b-4cf7-460d-a935-803a32c41ec6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'03c5f2a9-5802-48e8-8bc0-2495bcb3a9e3', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'277765e8-6602-482f-bfc9-0fe747acc6e0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ab00888d-d381-48e6-9208-de2017f1303d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'dd616365-614a-4bfa-ace5-482a590632dc', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'180d902a-4de5-4b2d-b97c-2acfbfc844f7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2e566e9b-8013-41fb-83ef-f30199a50238', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a4ca7bf5-e643-49df-819a-83de5a45e3f2', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1c23fdef-36e8-446b-bbef-1e09d87a8c21', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2bcb85c5-6701-47fa-b22c-793fd3dde2b7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8feb65e0-342a-4d6c-8a70-22dd5617c9ce', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ca760e43-db02-452f-b859-5a0dace55959', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'55e92ed7-6883-4684-9d55-f8b2dbbc5d0e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2ed06020-7028-4354-8113-a2b2a35938e6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ff4d6ef3-ecbf-4cd9-aba0-01dfae8a500e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ca685855-9a7d-4023-94a1-47d1d80203e8', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f168ed81-5362-4b29-bc39-f382ba4fe289', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'06d2fc12-e090-4ada-8a18-d34610515947', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f1a08018-de86-4d99-9a95-ed53cc80261b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd2069cb3-fbb1-494f-96e2-60bb59415004', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'de4c091f-f541-4eab-bcb9-434c77774c10', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'24a33c35-c7d1-4a41-a367-49c6d7ca8bbb', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'716435d4-f5aa-410f-91e8-e92028575d13', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e89cd330-44a0-42a2-8bc0-dfa836d0d4b2', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1120071e-4953-4a08-8a03-92e4f8bb72db', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'70ad5df3-e6b8-43b7-8d64-0d93480fe099', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8adf70f6-0437-47d5-ba45-91e50aab42cf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3021b8aa-5c61-4207-acb7-8b008e8f4451', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3af64e1e-926f-4b84-9192-00917af47d83', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f70e07a3-2a3b-46fa-b76c-a5b577548eee', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ccfa0974-d62e-44f4-be94-73379ef90d93', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'9a80a8de-3bf5-4694-af62-c7aac048e81b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ff3e7897-b81a-4956-afeb-dde87b58ef8c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'dcabcf39-ac9b-41fc-9c59-06ad578b32a6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'81c9ff7c-02a3-41b6-82da-9a1c662d3c59', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'31abd659-a171-4b51-8503-c2efb37fbe45', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8382b048-c0a1-4440-a5f5-d6b8e02f7d4a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'dd4269d9-f20d-4048-92ea-0efb9a491592', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2a416faf-c284-4614-b2a9-6438594a2f87', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'dcc9006c-05b6-4942-a897-315dda4f07b2', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'c48b0f75-3492-4143-b2cd-b02fed564965', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'55e84c5f-f66d-4071-8f7d-b3133e65fc48', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b75e996f-d893-4a5b-a062-6a616684cd7a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'44a106d8-ff6c-4faa-8034-d8bf98ffc030', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2536c17d-572a-478d-8457-0dc9ec0c1ad8', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'c7e0aebb-3b37-43df-817d-c9c6c8cfdb83', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5e3d22c6-456a-430d-afb3-5a1220311dad', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8ea40d0f-07d6-498e-8df2-95a14d57aca0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd25c0639-9274-4ff2-81a9-14c3bea53390', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e42e0df3-6f84-4921-9407-74574cee3022', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3dfa6983-dc80-46e9-bcce-59f6d3b83024', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd92fc8fe-2260-4b78-a01d-9da66deac004', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'543931c1-4de6-4761-bb34-5d2ec93a056a', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'75af64b7-6820-423a-9f9d-95c3979bfab5', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2efdd60f-3183-4d43-aea9-43c72ab43220', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'52792627-752c-4c21-b166-b636511e7a02', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'1e433055-444f-425e-bfa2-2ca4ed9a5e2e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'49e43eb9-b1a4-40c5-8de8-6b399ae89745', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b8fc56eb-e408-48b9-b423-8d9af294d065', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a10757eb-c683-4eca-8900-d09a4e373ebf', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e0d03e9c-80b5-431b-bc7d-9a780b396a3b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'01cc4f07-edb1-4353-bac1-81ba98e03283', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f51d98a3-ca7e-4589-831c-a1e8a796479f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'c0f5c106-683c-4262-bd2a-b141da844802', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'173e2281-de57-453a-bbe8-e533a2076e1c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fa6dd70e-1a26-481e-a7ab-7aab1d36a1d0', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'44a8f312-23c2-4a33-bf80-94b50fd7ead1', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'61c0cdd4-3b22-4977-841d-4348cae125fd', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'91bdf9d6-a7f9-4222-aa06-e29ffb4dfb77', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5ee0e413-7bf1-421d-842b-c3cca6457e1d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'862cae76-670a-4fbe-8a4e-106b229a156f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e3767cc6-a743-4e68-85af-28e91ec7f528', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5b93906b-7e8c-4368-a74c-6fd63c447f7c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e0a436ad-9b95-4b64-9a9f-2cbe6523beee', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'9e5e1628-fecc-48de-aab8-21919d8c66ab', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'43febf95-3400-4af0-95b9-0296051cf46f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3d60727f-c3b8-4ee4-baba-147064592cb6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'52e367d6-f2ff-43ff-b89c-95094b257e93', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'97139259-123c-411e-b1a4-7ad746be7116', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fe5e8f86-174f-4aec-b875-237c031eb54f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'34ed0dda-ba21-4368-9db0-c153fef5a789', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'60073bcd-7335-48af-9468-6ac048ff9a3f', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'24ca0b60-e9ab-484c-b6d1-7c9b1771226e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'731928b4-e680-4351-9657-b7e5f3329831', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ee696275-b601-4389-9ef4-027e7202ea63', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0b9e1fe5-466f-4bc4-a8a8-e587afeba789', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'7410b5a3-d54c-4c0e-b0ea-c71ddeb91197', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ae99aac6-f082-424c-8b14-c8be2d3bf6e4', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'72685bfe-3bc5-4c17-afcb-d725cbacd491', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'12af2b09-d135-4713-8db7-a1c990736a4e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'44e7740f-cf0d-4f7c-bc85-8c69c33faf2c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'febf2c58-6b79-4a77-a4e6-2d6c49676651', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'014d0f98-8bab-4917-a536-9fc7803bda67', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3652e0d3-e4fb-4f48-b736-23294d1ee094', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e0f66020-4fb0-481f-8c67-d540147b6d22', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'6e87a04a-d03b-466f-bb05-fae09bf11124', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'7f03f7a9-823f-4671-994f-f25c75566ebe', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'83f139b9-3db4-4881-a338-07b405ba25c9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'964a3e84-59c4-4657-8930-8f424a40f3b9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'43befc67-3d58-40ec-ac37-13244ffc1c75', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5d0e9923-4235-4ceb-a4fe-0910b6de6856', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'35ecf495-758b-49ea-aa9a-72fd12b616a7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'9b6139c6-9f1f-4c60-9816-a383fc14db69', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'a7b5f59d-f32a-4f8c-b7b4-8cf23a93fc74', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'28e46984-b161-4cd9-b746-9cbe5403771d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'b4627418-9d91-4e4e-b0cc-2d4db63c5f6e', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e726801f-77fa-4205-843f-f6bfea6ceca6', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'e0aa5b06-22a5-4c9b-8288-777670f256b7', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'063fdaf2-c007-4ddb-807e-2b9a0f2c02ba', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2f67cb76-2737-4099-8fc9-9461b8b8deec', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f391255c-ea3b-44b6-b075-06c8eca057b5', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'6a8ddec5-2d63-41a7-8707-085443ef37cd', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'16c84268-3061-467b-b018-7e55e951cfad', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ccca70bc-fbc0-4e33-bbf8-06db411603d9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'cecfcc85-c432-4e1c-9070-5baef355215d', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'65231c11-cb18-463e-87b3-e8e9615cf658', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'4ca41365-c5c8-40b4-9d72-f3e02b9a85cd', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f89b9460-498e-4f41-9f0c-a9c4c90cb7bd', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'63af8fc9-827d-4a6c-9bb7-6af5519fdb53', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f9052c97-00ce-44d5-a3db-6ed89dffe67b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'199c2c56-cc31-458c-94a0-503caf109c57', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'2e9fb4c1-1f16-4f35-b06b-415f64a93e0c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'440ee3de-8134-4488-b4fe-3f520383c0c9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3a51a4d8-638c-4404-8a82-e4a5d74d0f01', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'35bb1467-d204-4565-acfe-0d116e071009', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'ee3849e3-35fa-4ae1-a9ef-8eb8c4c07811', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'f2e44b88-9ade-4cf5-afb6-c957f961c790', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'8a64b572-60f6-4d2c-bd79-abf3781c467b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'fef01a4b-f3d4-4e9f-9770-4c36f386f898', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'71f58db7-8706-4f61-84b6-59e23b88c1b3', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd280b61f-ceb9-4088-8ff8-045ab532c2b5', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'24614c13-fb2c-4041-bae3-85808106b941', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'901752ca-26d7-4854-9216-374f9534cdde', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5c489084-872e-4164-b0e4-5a5d366f5f46', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'd0e77720-5494-4a66-b892-7d46fd59f3b9', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'5ff48ad5-8976-4cf8-916a-eae8d48b561c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'14ab79c1-a7e2-4a9d-9216-7017b0487b7b', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'0c9f1e08-5fd7-4703-af0d-05ac4474cb79', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'3f169469-2ae3-4e8e-ae39-16cf9e440a8c', u'4DNFIO67AFHV')
(u'md5', 'deleted file workflow', u'693f3694-ec27-4ed9-b841-96b09dd13cec', u'4DNFIO67AFHV')
(u'hi-c-processing-partc', 'old style or dub', u'8c8952cb-e828-4362-bccb-e7cad740f11b', u'4DNFIU59TRDY')
(u'hi-c-processing-partc', 'old style or dub', u'3764050b-c389-4000-998b-ee625f93e876', u'4DNFIU59TRDY')
(u'hi-c-processing-partc', 'old style or dub', u'8e4a7dcb-e7ee-4580-a43a-f72af23408b5', u'4DNFIU59TRDY')
(u'hi-c-processing-partc', 'old style or dub', u'8c8952cb-e828-4362-bccb-e7cad740f11b', u'4DNFIHKCL9SF')
(u'hi-c-processing-partc', 'old style or dub', u'3764050b-c389-4000-998b-ee625f93e876', u'4DNFIHKCL9SF')
(u'hi-c-processing-partc', 'old style or dub', u'8e4a7dcb-e7ee-4580-a43a-f72af23408b5', u'4DNFIHKCL9SF')
(u'hi-c-processing-partc', 'old style or dub', u'01b65168-176d-4657-95af-e56220a6a8fa', u'4DNFIAQWZYR5')
(u'hi-c-processing-partc', 'old style or dub', u'01b65168-176d-4657-95af-e56220a6a8fa', u'4DNFILYFQZ3S')
200 5
(u'hi-c-processing-partc', 'old style or dub', u'755db506-0aa8-4e10-919b-4f46273f8742', u'4DNFIUVPBSQO')
300 6
(u'hi-c-processing-partb', 'old style or dub', u'e0e64243-9e9b-4bc1-8d5a-03ddeb3c1268', u'4DNFI6R0M6JF')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFI6R0M6JF')
(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFI6R0M6JF')
(u'hi-c-processing-partb', 'old style or dub', u'e0e64243-9e9b-4bc1-8d5a-03ddeb3c1268', u'4DNFIF6BRK3Z')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIF6BRK3Z')
(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFIF6BRK3Z')
(u'hi-c-processing-partb', 'old style or dub', u'e0e64243-9e9b-4bc1-8d5a-03ddeb3c1268', u'4DNFIP5YPJ6C')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIP5YPJ6C')
(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFIP5YPJ6C')
(u'hi-c-processing-partb', 'old style or dub', u'e0e64243-9e9b-4bc1-8d5a-03ddeb3c1268', u'4DNFIV6KEBIF')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIV6KEBIF')
(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFIV6KEBIF')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIANXLTA3')
(u'hi-c-processing-partb', 'old style or dub', u'd9a6a8c9-3db3-4c7c-9986-f07948adbc27', u'4DNFIANXLTA3')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIMDUVALM')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIX97V4QW')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIXDYZNGA')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIY4ADY12')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIAI5WWUK')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIRXWQHY8')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIU1MIHUL')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIHCWUJV2')
(u'hi-c-processing-partb', 'old style or dub', u'3fe79fd4-4c80-4fce-bbc2-f50f4d0b45af', u'4DNFIJ94VPJ5')
400 20
(u'hi-c-processing-partb', 'old style or dub', u'ca5769f1-f12d-49cc-b6e9-c08af463f0e5', u'4DNFIFTJPJMT')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFIFTJPJMT')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFIFTJPJMT')
(u'hi-c-processing-partb', 'old style or dub', u'034dcf4e-dce3-4946-b31c-84e5dbe79a36', u'4DNFIFTJPJMT')
(u'hi-c-processing-partb', 'old style or dub', u'ca5769f1-f12d-49cc-b6e9-c08af463f0e5', u'4DNFIMYJGXY6')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFIMYJGXY6')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFIMYJGXY6')
(u'hi-c-processing-partb', 'old style or dub', u'034dcf4e-dce3-4946-b31c-84e5dbe79a36', u'4DNFIMYJGXY6')
(u'hi-c-processing-partb', 'old style or dub', u'ca5769f1-f12d-49cc-b6e9-c08af463f0e5', u'4DNFI5GMT4U1')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFI5GMT4U1')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFI5GMT4U1')
(u'hi-c-processing-partb', 'old style or dub', u'034dcf4e-dce3-4946-b31c-84e5dbe79a36', u'4DNFI5GMT4U1')
(u'hi-c-processing-partb', 'old style or dub', u'ca5769f1-f12d-49cc-b6e9-c08af463f0e5', u'4DNFIABMIHH0')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFIABMIHH0')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFIABMIHH0')
(u'hi-c-processing-partb', 'old style or dub', u'034dcf4e-dce3-4946-b31c-84e5dbe79a36', u'4DNFIABMIHH0')
(u'hi-c-processing-partb', 'old style or dub', u'ca5769f1-f12d-49cc-b6e9-c08af463f0e5', u'4DNFIKEN54CP')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFIKEN54CP')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFIKEN54CP')
(u'hi-c-processing-partb', 'old style or dub', u'00336b14-262d-4765-8e2d-764e2f0a8287', u'4DNFIIAGZST8')
(u'hi-c-processing-partb', 'old style or dub', u'2a1b6b9e-b13b-4c38-82b6-962c527c0d40', u'4DNFIIAGZST8')
500 26
(u'hi-c-processing-partb', 'old style or dub', u'cdc2e75a-2d6b-4b41-b651-5a9e8b9f5926', u'4DNFIY5V5MYI')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFIY5V5MYI')
(u'hi-c-processing-partb', 'old style or dub', u'cdc2e75a-2d6b-4b41-b651-5a9e8b9f5926', u'4DNFIPJQ4V37')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFIPJQ4V37')
(u'hi-c-processing-partb', 'old style or dub', u'cdc2e75a-2d6b-4b41-b651-5a9e8b9f5926', u'4DNFIEXCW3TS')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFIEXCW3TS')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFI1C9RJGI')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFIIQN4FKO')
(u'hi-c-processing-partb', 'old style or dub', u'cdc2e75a-2d6b-4b41-b651-5a9e8b9f5926', u'4DNFIFCXCE1W')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFIFCXCE1W')
(u'hi-c-processing-partb', 'old style or dub', u'cdc2e75a-2d6b-4b41-b651-5a9e8b9f5926', u'4DNFI9U1FBR2')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'5523499f-3c53-4ee7-a635-1de2417216d4', u'4DNFI9U1FBR2')
(u'hi-c-processing-partb', 'old style or dub', u'3bb84eba-aded-41b1-8294-6801ff7831ed', u'4DNFI3925W5X')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'e9c7dcca-f3b2-4a5b-a061-5b6ae18bae17', u'4DNFI3925W5X')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFI97C92I8')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFI97C92I8')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIV5ZJ2W2')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIV5ZJ2W2')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFI5BJFANX')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFI5BJFANX')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIB92E1OR')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIB92E1OR')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIMGDP12P')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIMGDP12P')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIS1Z4CI8')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIS1Z4CI8')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIZYHSUUX')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIZYHSUUX')
(u'hi-c-processing-partb', 'old style or dub', u'5ddac3e8-22d0-4f0f-b5a4-a8d4c2a21781', u'4DNFIGYY2YMF')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'a3846fd9-3549-4989-bbf7-40475ea908d6', u'4DNFIGYY2YMF')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI0512V8H')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIABQZVC2')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFII6531N3')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIYL2VHZZ')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIM41FHD5')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIJF0BKCZ')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIALS22LU')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIK3EPUGD')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI1XQBFUP')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI7UX6K5S')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI2X3ORY5')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFI5H0U2IB')
600 54
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIKKZBHI1')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFILJJ6PMI')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIC6HGO0T')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIASXKH2N')
('saved from deletion', u'hi-c-processing-partb', 'old style or dub', u'0970eba6-2be9-4a7d-a48a-75fea1742b76', u'4DNFIVU1YKV3')
700 59
800 59
900 59
1000 59
1100 59
1200 59
1300 59
1400 59
(u'fastqc-0-11-4-1', 'old style or dub', u'd70b4599-658a-4020-9430-1687ac87f973', u'4DNFIK96H3DL')
(u'fastqc-0-11-4-1', 'old style or dub', u'8abcbf20-c2bb-4a8e-b0c3-3411a4a4e406', u'4DNFINPFR88E')
1500 61
(u'fastqc-0-11-4-1', 'old style or dub', u'40111d03-6388-42af-a61a-c64b3915cfe2', u'4DNFIZAIE1P2')
(u'fastqc-0-11-4-1', 'old style or dub', u'38edde43-e4db-4247-a8da-0107f2daf838', u'4DNFIKZUVWKB')
(u'fastqc-0-11-4-1', 'old style or dub', u'f5965a6c-cb42-43da-96e8-e7168b2b8edc', u'4DNFIQNT4H9T')
1600 64
(u'fastqc-0-11-4-1', 'old style or dub', u'591b8160-6ddb-43bf-aa26-82bf677f82b7', u'4DNFIDGAVP7C')
(u'fastqc-0-11-4-1', 'old style or dub', u'52bd6715-d63b-4263-a21d-7b7066911316', u'4DNFIZ5HEG3B')
(u'fastqc-0-11-4-1', 'old style or dub', u'73d1f6a2-d4b8-440a-b1f1-9bbc74f4fcf4', u'4DNFILFI89TD')
(u'fastqc-0-11-4-1', 'old style or dub', u'34df5082-56d1-44e3-82ca-dd248bb74d32', u'4DNFIZ988CQG')
(u'fastqc-0-11-4-1', 'old style or dub', u'eb01c9a8-9f6f-4130-8cc7-c902c6f0fbc0', u'4DNFIQC9Y1QC')
(u'fastqc-0-11-4-1', 'old style or dub', u'9eed6113-9e8a-4b19-be78-68ff32b8659e', u'4DNFIFG2VLFU')
(u'fastqc-0-11-4-1', 'old style or dub', u'67051b18-88c4-4b81-bed9-04350634f328', u'4DNFIWF658RG')
(u'fastqc-0-11-4-1', 'old style or dub', u'027e8d52-ee93-4d8d-82c6-4fe72c333a7f', u'4DNFI6FBG681')
1700 72
(u'fastqc-0-11-4-1', 'old style or dub', u'5b2fe18c-ef3a-4580-bdb9-096b40e6e86a', u'4DNFIW83E59D')
(u'fastqc-0-11-4-1', 'old style or dub', u'41dd21ab-d1ad-4a97-90a1-a76c7381821e', u'4DNFIX1J1O73')
(u'fastqc-0-11-4-1', 'old style or dub', u'4c13a741-7e04-4813-9930-7809c7572a15', u'4DNFIKXBLLA2')
(u'fastqc-0-11-4-1', 'old style or dub', u'c23f53c6-c66c-48e6-8694-7a1012454872', u'4DNFIMZQFHM2')
1800 76
(u'fastqc-0-11-4-1', 'old style or dub', u'206b206c-39c3-4615-8231-49368752afc4', u'4DNFI1PNX9UU')
1900 77
(u'fastqc-0-11-4-1', 'old style or dub', u'ef322731-0cd2-49f0-9523-0cfbe9429f85', u'4DNFIKEHEQLM')
2000 78
(u'fastqc-0-11-4-1', 'old style or dub', u'6024e69d-aa08-482a-b5b4-a7ded929e3c1', u'4DNFI6WJA6CN')
2100 79
(u'md5', 'old style or dub', u'0bcda58c-4097-4371-9bec-ba4cd81d73a3', u'4DNFILXL5PL6')
(u'md5', 'old style or dub', u'ddefa0ec-eba5-47fd-b7db-cbd09afba5be', u'4DNFIJ5LMO1J')
(u'md5', 'old style or dub', u'1b5c5299-78ee-4273-a749-8ce49531c9b2', u'4DNFI46IF649')
(u'fastqc-0-11-4-1', 'old style or dub', u'4c551b4e-ad58-4666-b8cb-9586540868b4', u'4DNFIN9VCK6B')
(u'md5', 'old style or dub', u'3a82e51f-c977-427b-bd65-704e36ad3536', u'4DNFI85FACK4')
(u'md5', 'old style or dub', u'9ed98fc0-f7d2-4cbd-8f3d-108d232e57a6', u'4DNFIMRO733N')
(u'md5', 'old style or dub', u'99cf1ea3-63b9-43f5-8876-c7df0ada570b', u'4DNFIX5324L6')
(u'md5', 'old style or dub', u'8f642cb5-c628-4e73-8c7c-baf880b9bc8a', u'4DNFIRO7T5YL')
(u'md5', 'old style or dub', u'5d439809-10aa-456b-9564-b65255fcf72c', u'4DNFIEHZX2N8')
(u'md5', 'old style or dub', u'33019258-a8d4-472b-8b52-ac2e1d7cd7c0', u'4DNFI1GKYC57')
(u'md5', 'old style or dub', u'88f2cc3a-42ae-4240-bcc7-691aac46142c', u'4DNFI9TGTTYY')
(u'md5', 'old style or dub', u'9aff97e1-13c5-47ee-b46d-7c0515950dd0', u'4DNFIY3ST1MI')
(u'md5', 'old style or dub', u'9ca0bcf0-a32a-40cd-81be-dfeed228110f', u'4DNFIBD2PO52')
(u'md5', 'old style or dub', u'dc13e58a-b034-442c-87e4-ee91765b0422', u'4DNFINPGLVF6')
2200 93
(u'fastqc-0-11-4-1', 'old style or dub', u'2e48f680-469f-4eb5-bf4e-4058941522fa', u'4DNFIULWI1B3')
2300 94
(u'fastqc-0-11-4-1', 'old style or dub', u'12879f94-31f3-40bc-b6b0-5ebbf0fa32fb', u'4DNFIRGIBV63')
(u'fastqc-0-11-4-1', 'old style or dub', u'bf678fcd-0fc0-4f3c-9dde-0653b549b681', u'4DNFIRGIBV63')
(u'md5', 'old style or dub', u'194bc90f-b912-4c2c-9564-d8b8c8aac718', u'4DNFIMJCWJ4O')
(u'md5', 'old style or dub', u'ba02bfb9-899e-4894-aece-66995e71b951', u'4DNFIPJVFYSX')
(u'md5', 'old style or dub', u'e04ff630-010b-40b9-b6bb-43780ca4598f', u'4DNFI75KWVST')
(u'md5', 'old style or dub', u'3d95d998-9098-46f8-848d-c87cb87334a0', u'4DNFIRJ8G68S')
(u'md5', 'old style or dub', u'a4d9d807-11d2-46f1-8eed-ef256eb8d5da', u'4DNFI1R3E54X')
2400 100
(u'md5', 'old style or dub', u'f7ef3cf8-b921-4171-97eb-e602ff896bf8', u'4DNFIV5MDEUU')
(u'md5', 'old style or dub', u'95695f92-bde1-43ab-b384-0ce995014c46', u'4DNFIBAKK7QS')
(u'md5', 'old style or dub', u'b29690c7-8991-47dc-989f-bd9738858eba', u'4DNFIYFF7KJY')
(u'md5', 'old style or dub', u'72c60f26-9741-48e5-aa62-c3b2faf98306', u'4DNFIR9GJWST')
(u'md5', 'old style or dub', u'35355c62-df33-4df1-a383-c0c8acaa4393', u'4DNFIKQLSC9T')
(u'md5', 'old style or dub', u'869b593c-5890-4462-b057-0da1e3b27e07', u'4DNFI44JD4EE')
(u'md5', 'old style or dub', u'fdf66199-476d-4b57-9056-26baca31c8af', u'4DNFIW7FKQXS')
(u'md5', 'old style or dub', u'303ad507-e3d4-4b5b-bf7e-c22280bb1dd8', u'4DNFIWV5A3Z6')
(u'md5', 'old style or dub', u'15998296-8dac-4b52-8040-047e3a5283c9', u'4DNFIGP82LZH')
(u'md5', 'old style or dub', u'9a9816e3-cea8-4127-b012-03ad4d465832', u'4DNFILGIAEZD')
(u'md5', 'old style or dub', u'8f8dc8fe-0af1-4c8e-b288-0a476e619bf9', u'4DNFIB94UMHJ')
(u'md5', 'old style or dub', u'3f288355-993b-4c4f-90fa-9224142cc5b8', u'4DNFIASHJZCY')
2500 112
(u'md5', 'old style or dub', u'fb2266ce-1c53-4ad0-88f3-a2ce192829be', u'4DNFIH2FMKX4')
(u'md5', 'old style or dub', u'51ecd3ea-dae9-4545-af4a-21a1c441097c', u'4DNFI5BO2H1Z')
(u'md5', 'old style or dub', u'c2408275-16d3-48d8-9f13-71e2380dba3b', u'4DNFIZ5X4QH6')
(u'md5', 'old style or dub', u'b2dc0f5c-cff4-4367-b9e2-c9eb9a2a7746', u'4DNFI7T43MJ6')
(u'md5', 'old style or dub', u'81824168-e69f-4389-ba8b-5a53c1d412a5', u'4DNFIBYV2SS8')
(u'md5', 'old style or dub', u'911d58a1-0587-4d15-bf32-d7227e1c4c23', u'4DNFI34MGS31')
(u'md5', 'old style or dub', u'c092077e-a4dd-428f-a34e-32e05452e95e', u'4DNFI3VYO28H')
(u'md5', 'old style or dub', u'86f075ed-6f15-4121-8674-3ec8da243e0c', u'4DNFIT48F7C7')
(u'md5', 'old style or dub', u'645841e5-9e24-4ec3-b79c-ae63032cef09', u'4DNFIAU6QDBL')
(u'md5', 'old style or dub', u'e0cc1f68-1521-4b47-ac2f-b21cc59fffd5', u'4DNFIYTLTSFN')
(u'md5', 'old style or dub', u'3300fc3d-da98-4da6-8129-b1c5b892146b', u'4DNFIN5YSD9A')
(u'md5', 'old style or dub', u'215b9827-416b-46da-9b74-f4e50d0bc48c', u'4DNFIBF377E2')
(u'md5', 'old style or dub', u'11fb40c2-e0be-434b-b0ef-e99d3c383a17', u'4DNFIPVUH4E5')
(u'md5', 'old style or dub', u'4a1cfe1e-84ef-4c4e-96d4-74a326dd3d58', u'4DNFIEPVXYTH')
(u'md5', 'old style or dub', u'a6e2379b-7233-4b7c-97fd-31cc862d7106', u'4DNFIOA5TNC1')
(u'md5', 'old style or dub', u'2a25c752-7eb3-4e6f-8544-a58f537ed90a', u'4DNFIJPD156A')
(u'md5', 'old style or dub', u'83942ec1-f119-4751-a1d8-f180adb1041e', u'4DNFIJPD156A')
(u'md5', 'old style or dub', u'5d61f6fd-c272-47cc-ae4a-f25c8c9036d3', u'4DNFIZHUKESO')
(u'md5', 'old style or dub', u'acd00048-e4e1-497a-b9c9-f141cc4f93d0', u'4DNFIM7JG41M')
(u'md5', 'old style or dub', u'be9a7b55-6904-4a5d-8440-9faa47fec2ed', u'4DNFIMGO4BQX')
(u'md5', 'old style or dub', u'e268ea07-5a2f-480f-95f6-2a784fe96ed7', u'4DNFIMVAKKBC')
2600 132
2700 132
2800 132
2900 132
(u'hi-c-processing-partb', 'deleted file workflow', u'e5927906-cf80-46da-8a81-1093926865af', u'4DNFIN232ZZZ')
(u'hi-c-processing-partb', 'deleted file workflow', u'5cc47020-2397-46c7-975b-a0c19b2899fd', u'4DNFIN232ZZZ')
(u'hi-c-processing-partb', 'deleted file workflow', u'e5927906-cf80-46da-8a81-1093926865af', u'4DNFPM172I2T')
(u'hi-c-processing-partb', 'deleted file workflow', u'5cc47020-2397-46c7-975b-a0c19b2899fd', u'4DNFPM172I2T')
(u'hi-c-processing-partb', 'deleted file workflow', u'e5927906-cf80-46da-8a81-1093926865af', u'4DNFHFYGN09R')
(u'hi-c-processing-partb', 'deleted file workflow', u'5cc47020-2397-46c7-975b-a0c19b2899fd', u'4DNFHFYGN09R')
3000 135
(u'md5', 'old style or dub', u'9de58daf-1c83-4f3d-81e5-e08034a65676', u'4DNFIN1VFFCD')
(u'md5', 'old style or dub', u'fec40be6-f565-405f-9b74-d8754274685e', u'4DNFIQ5NI8TY')
(u'md5', 'old style or dub', u'0961c843-ada5-4abb-8ef1-9f66a286faa7', u'4DNFIQK4MFWL')
(u'md5', 'old style or dub', u'5f0be557-782f-40b7-8448-4e1c3b92c815', u'4DNFI37UIVDH')
(u'md5', 'old style or dub', u'332088d0-7d28-419f-916b-40d1c4a59e5a', u'4DNFIA8HTWVW')
(u'md5', 'old style or dub', u'629bd164-e1d5-4f72-877b-4f7284df3148', u'4DNFIHCMAFAZ')
(u'md5', 'old style or dub', u'521a9000-6782-43db-8b6c-ff770f8c40bf', u'4DNFITVNI13Q')
(u'md5', 'old style or dub', u'be823ac0-d75e-4b09-8ae2-1e58fe71afc9', u'4DNFIZGJG2VU')
(u'md5', 'old style or dub', u'7e98ddcc-8e40-4e5a-8853-edae9f670ce3', u'4DNFIGTAI53P')
(u'md5', 'old style or dub', u'57266800-fbc9-4db8-97fe-8dfd40124cfe', u'4DNFI6W7Y783')
(u'md5', 'old style or dub', u'91ace2af-b1c6-4cf9-abc5-7e50343e5549', u'4DNFIEN6IM4S')
(u'md5', 'old style or dub', u'b3421124-0dc6-4c49-99e5-98f91b952196', u'4DNFIPSKHIBT')
(u'md5', 'old style or dub', u'a846fa38-04d5-4e7f-8fd8-70355fec0988', u'4DNFIXPNTARQ')
(u'md5', 'old style or dub', u'577e7daa-5ae5-4a56-96d3-88b14297480c', u'4DNFI253EY3I')
(u'md5', 'old style or dub', u'2afbc6f6-337a-4265-bbab-3de5079d46e8', u'4DNFIX3X22EQ')
(u'fastqc-0-11-4-1', 'old style or dub', u'f460de40-cfb3-494c-9520-0735b1e81557', u'4DNFIQ4VTRCE')
(u'md5', 'old style or dub', u'104e90ec-9863-487c-8b6a-d671fa5612df', u'4DNFIBPSVSZ6')
(u'fastqc-0-11-4-1', 'old style or dub', u'596b036b-2fdd-42d2-aa9d-bce3a6912c11', u'4DNFIBPSVSZ6')
(u'md5', 'old style or dub', u'62964dc0-42e0-45b4-afa5-fde1d1dea2fa', u'4DNFI65W1NWV')
3100 153
(u'md5', 'old style or dub', u'24be4615-5cce-428a-a1ad-6eea58366923', u'4DNFICU9DO7Z')
(u'md5', 'old style or dub', u'700aada3-6e7a-458f-ab79-7de06269da6d', u'4DNFI4USEWYJ')
(u'md5', 'old style or dub', u'b6c0c872-598e-4a3c-8ee2-6a8983acdff8', u'4DNFIB1JOP3F')
(u'md5', 'old style or dub', u'6e3a2c6f-bd58-41dd-8ed9-ca566175bffd', u'4DNFI9Y415AI')
(u'md5', 'old style or dub', u'0b8db59f-b29d-4457-9227-a589ccf66666', u'4DNFIZDWZKW6')
(u'md5', 'old style or dub', u'2e13dd6f-a8b4-4ef8-a9a3-22556db3554c', u'4DNFICSBWWWW')
(u'md5', 'old style or dub', u'4aa05b0a-80b3-48b6-bbde-7d0622748ab6', u'4DNFIST7XI9H')
(u'md5', 'old style or dub', u'093c3458-5aa4-47fa-9262-bdab2ce2775c', u'4DNFIDK996IR')
(u'md5', 'old style or dub', u'f4cd1d65-cdcf-4429-a265-9e29e2c9c90e', u'4DNFIGHF2VDE')
(u'md5', 'old style or dub', u'5e0c782e-0430-4c22-95c1-b213e604dcf7', u'4DNFIK1LWQ54')
(u'md5', 'old style or dub', u'67b97487-50e2-4460-a068-c7dd456b82f5', u'4DNFIK1LWQ54')
(u'md5', 'old style or dub', u'8c663d71-df39-434a-a0fd-897646b3ecab', u'4DNFIP1SKFKA')
(u'md5', 'old style or dub', u'24accb3c-75d9-4444-b146-7ea1df71942b', u'4DNFIDSCVHA4')
(u'md5', 'old style or dub', u'576932d2-dcff-41f3-bb34-b0075c496448', u'4DNFIO7DLI48')
(u'md5', 'old style or dub', u'c298f6e7-093a-4ff5-a624-3e12814666e2', u'4DNFIUY8JQ8M')
(u'md5', 'old style or dub', u'26730b50-221c-462a-a126-958aa0fe9b2b', u'4DNFIKECI2BJ')
(u'md5', 'old style or dub', u'a1984349-3717-43c3-a2d3-31f5705c1a7a', u'4DNFI74MOQ66')
(u'md5', 'old style or dub', u'4276c2f3-85b9-4fca-b412-80e8575e07bd', u'4DNFIMOXQ9UR')
(u'md5', 'old style or dub', u'fffbc76a-37a9-4b5a-8b09-b8667db5a90e', u'4DNFIYD2XMBA')
(u'md5', 'old style or dub', u'4e882bdc-d7f0-46e3-81e0-a2cda2f1c638', u'4DNFILT5X7TK')
(u'md5', 'old style or dub', u'9d028cee-4d1d-45b7-adf9-24a0b682f317', u'4DNFI8W9YL4S')
(u'md5', 'old style or dub', u'8e9c2baf-aa4a-48f3-b05f-cf12a39db22e', u'4DNFIRENAFZ3')
(u'md5', 'old style or dub', u'3e80f505-5c85-4a34-b7f9-295e0db4ae83', u'4DNFI6PUKY6I')
(u'md5', 'old style or dub', u'aeb37e32-82d1-4c5c-8359-82e1bb044ca4', u'4DNFIMTPBZY9')
(u'md5', 'old style or dub', u'83ba33a0-ec1d-47d1-a78e-53f17307ea0b', u'4DNFIMTPBZY9')
(u'md5', 'old style or dub', u'29c2e19e-58af-4105-af15-be8974115508', u'4DNFIIQVPD73')
(u'md5', 'old style or dub', u'fc197542-9748-48e1-b315-2ee15d2f722a', u'4DNFIEGXB7QA')
(u'md5', 'old style or dub', u'6991fd92-2837-4043-93d3-6ffa8b9fc8d7', u'4DNFI6J2WOO2')
(u'md5', 'old style or dub', u'5ae53da6-6d46-4c78-8665-6144897b2121', u'4DNFIRUPDEXP')
(u'md5', 'old style or dub', u'b27417d2-b303-4570-a1bb-e2612c28375f', u'4DNFIZ44BASO')
(u'md5', 'old style or dub', u'0731658f-9dab-4155-b1da-d965d04db204', u'4DNFITL1BN7A')
(u'md5', 'old style or dub', u'44f7f7be-34f8-4dae-94db-edd0ccfaed14', u'4DNFIAJK2TE4')
(u'md5', 'old style or dub', u'68facb8b-57f5-4d87-95c9-e0bf7352b887', u'4DNFI2QXK1GU')
(u'md5', 'old style or dub', u'af9aee27-7394-4ac6-941f-6cd4cf0f118a', u'4DNFIR8778O4')
3200 185
(u'md5', 'old style or dub', u'ed63b7c5-593f-4b65-81b9-3df8fade2080', u'4DNFI4DWD88F')
(u'md5', 'old style or dub', u'78dc32a0-b753-4b65-9937-1e49b581f99e', u'4DNFI4DWD88F')
(u'md5', 'old style or dub', u'ae3a4a3b-506e-4588-b227-213d00251c2d', u'4DNFI8LZYRSF')
(u'md5', 'old style or dub', u'79b88cbd-2123-48c5-8c92-6bc2ec5917f3', u'4DNFIZ7G79A9')
(u'md5', 'old style or dub', u'5952a12a-d2e6-403e-9326-ea65721b40b2', u'4DNFIB1Y8SXF')
(u'md5', 'old style or dub', u'2c1e1ca7-53f0-4342-a065-cd74b05a7f87', u'4DNFIHHGTS7L')
(u'md5', 'old style or dub', u'ee18d15f-d23b-4d41-a12f-994e47b18cc9', u'4DNFIL4QYF26')
(u'md5', 'old style or dub', u'2c530e24-22a0-4988-bd45-3c2b4c49082a', u'4DNFI23CNILG')
3300 192
3400 192
3500 192
3600 192
3700 192
3800 192
3900 192
4000 192
4100 192
4200 192
4300 192
482 workflowruns from 192 files deleted
449
3 md5 fields deleted
1 qc metrics deleted
279 deleted output files
finished at 2017-09-12 20:32:30.120819